# Import Necessary Libraries and modules

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,BatchNormalization,Flatten,Activation
from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.utils import plot_model
import seaborn as sns

# Directories of the datasets


In [ ]:
train_directory='../input/100-bird-species/train'
val_directory='../input/100-bird-species/valid'
test_directory='../input/100-bird-species/test'

# Rescaling the Images

In [ ]:
train_datagen=ImageDataGenerator(rescale=1/255)
val_datagen=ImageDataGenerator(rescale=1/255)
test_datagen=ImageDataGenerator(rescale=1/255)

# Using the flow from directory function for reading the images

In [ ]:
train_generator=train_datagen.flow_from_directory(train_directory,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                  batch_size=256,
                                                 class_mode='sparse')
val_generator=val_datagen.flow_from_directory(val_directory,
                                                 target_size=(224,224),
                                                 batch_size=256,
                                                 color_mode='rgb',
                                                 class_mode='sparse')
test_generator=test_datagen.flow_from_directory(test_directory,
                                                batch_size=256,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 class_mode='sparse')

# The Dictionary

In [ ]:
num_classes=(len(train_generator.class_indices))

In [ ]:
train_generator.class_indices

# Understanding the Data

In [ ]:
breeds=list(train_generator.class_indices.keys())

In [ ]:
x=list(train_generator.classes)

In [ ]:
label=[]
for i in range (0,225):
    label.append(x.count(i))


# Plotting the Distribution Graphs

In [ ]:
fig_dims = (15, 4)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x=breeds[:60],y=label[:60])
plt.xticks(rotation=90)
plt.show()

In [ ]:
fig_dims = (15, 4)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x=breeds[61:120],y=label[61:120])
plt.xticks(rotation=90)
plt.show()

In [ ]:
fig_dims = (17, 5)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x=breeds[121:200],y=label[121:200])
plt.xticks(rotation=90)
plt.show()

In [ ]:
fig_dims = (10, 4)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x=breeds[201:],y=label[201:])
plt.xticks(rotation=90)
plt.show()

# Plotting Edges using ImageJ

In [ ]:
image.load_img('../input/edges-birds/american cootedges.jpg')

In [ ]:
image.load_img('../input/edges-birds/antbirdedges.jpg',target_size=(360,480))

In [ ]:
image.load_img('../input/edges-birds/bald eagle edges.jpg',target_size=(360,480))

In [ ]:
image.load_img('../input/edges-birds/kookaburra edges.jpg',target_size=(360,480))

In [ ]:
image.load_img('../input/edges-birds/razor billedges.jpg',target_size=(360,480))

# Using Resnet50v2 with imagenet weights

In [ ]:
from keras.applications import ResNet50V2

In [ ]:
convlayer=ResNet50V2(input_shape=(224,224,3),weights='imagenet',include_top=False)
for layer in convlayer.layers:
    layer.trainable=False

In [ ]:
plot_model(convlayer)

# Adding the layer to the Dense Layers

In [ ]:
model=Sequential()
model.add(convlayer)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax',kernel_initializer='glorot_uniform'))
print(model.summary())

In [ ]:
plot_model(model)

# Use a small learning rate for better accuracy

In [ ]:
optimizer=keras.optimizers.RMSprop(lr=0.0001)
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,epochs=7,validation_data=val_generator)

In [ ]:
plt.plot(history.history['accuracy'],c='b',label='train')
plt.plot(history.history['val_accuracy'],c='r',label='validation')
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'],c='b',label='train')
plt.plot(history.history['val_loss'],c='r',label='validation')
plt.legend(loc='upper right')
plt.show()

# Check your Accuracy on the Test set

In [ ]:
model.evaluate(test_generator)

# Check for Your Own Image

In [ ]:
def predict(dir):
    img=image.load_img(dir,target_size=(224,224))
    img1=image.img_to_array(img)
    img1=img1/255
    img1=np.expand_dims(img1,[0])
    plt.imshow(img)
    pred=int((model.predict_classes(img1)))
    for key,values in train_generator.class_indices.items():
        if pred==values:
            print("The predicted image of the bird is: "+str(key)+" with a probability of "+str(round(np.max(model.predict_proba(img1))*100,2))+"%")
    
    

# Example1

In [ ]:
predict('../input/bird-custom-check-data/AmericanKestrel.jpg')

# Example2

In [ ]:
predict('../input/bird-custom-check-data/american coot.jpg')

# Example 3

In [ ]:
predict('../input/bird-custom-check-data/anhinga.jpg')

# Example 4

In [ ]:
predict('../input/bird-custom-check-data/antbird.jpg')

# Example 5

In [ ]:
predict('../input/bird-custom-check-data/bald eagle.jpg')

# Example 6

In [ ]:
predict('../input/bird-custom-check-data/baltimore oriole.jpg')

# Example 7

In [ ]:
predict('../input/bird-custom-check-data/cassowary.jpg')

# Example 8

In [ ]:
predict('../input/bird-custom-check-data/golden winged warber.jpg')

# Example 9

In [ ]:
predict('../input/bird-custom-check-data/kookaburra.jpg')

# Example 10

In [ ]:
predict('../input/bird-custom-check-data/osprey.jpg')

# Example 11

In [ ]:
predict('../input/bird-custom-check-data/woodduck.jpg')

# Example 12

In [ ]:
predict('../input/bird-custom-check-data/incatern.jpg')